# **Cervical Spine Detection Solution Part 3**

In [ ]:
 !pip install -qU "python-gdcm" pydicom pylibjpeg "opencv-python-headless"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot, plot
from tqdm.notebook import tqdm
from pathlib import Path
from collections import Counter

tqdm.pandas()

In [ ]:
import pydicom
import nibabel as nib
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
paths = {
    'train_df': Path('../input/rsna-2022-cervical-spine-fracture-detection/train.csv'),
    'train_bbox': Path('../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv'),
    'train_images': Path('../input/rsna-2022-cervical-spine-fracture-detection/train_images'),
    'train_nifti_segments': Path('../input/rsna-2022-cervical-spine-fracture-detection/segmentations'),
    'test_df': Path('../input/rsna-2022-cervical-spine-fracture-detection/test.csv'),
    'test_images': Path('../input/rsna-2022-cervical-spine-fracture-detection/test_images')
}


In [ ]:
train_df = pd.read_csv(paths['train_df'])
test_df = pd.read_csv(paths['test_df'])

In [ ]:
train_df.head()

In [ ]:
train_df[train_df['StudyInstanceUID'] == '1.2.826.0.1.3680043.10443']

In [ ]:
train_df.drop(index=1135, inplace=True)
train_df.reset_index(drop=True, inplace=True)

In [ ]:
train_df.head()

In [ ]:
train_df['total_fractures'] = train_df.loc[:,[f"C{i}" for i in range(1,8)]].sum(axis=1)

In [ ]:
len(train_df)

In [ ]:
train_df['segment_path'] = train_df['StudyInstanceUID'].map(lambda x: paths['train_images']/x)

In [ ]:
def num_slices(path):
    slices = list(path.glob('*'))
    return len(slices)

train_df['num_slices'] = train_df['segment_path'].progress_map(num_slices)
train_df['num_slices'] = train_df['num_slices'].astype('int')

In [ ]:
train_df.head()

In [ ]:
random_dcm_file = list(train_df['segment_path'][123].glob('*'))[0]
print(random_dcm_file)
random_dcm_file = pydicom.dcmread(random_dcm_file)
print(random_dcm_file)

In [ ]:
print("Instance Number:",random_dcm_file.get('InstanceNumber'))
print("Rows x Columns:", random_dcm_file.get("Rows"), random_dcm_file.get("Columns"))
print("Image Position (Patient):", random_dcm_file.get("ImagePositionPatient"))

In [ ]:
dcm_image = apply_voi_lut(random_dcm_file.pixel_array, random_dcm_file)

In [ ]:
plt.imshow(dcm_image, cmap='bone')

In [ ]:
random_nii_file = list(paths['train_nifti_segments'].glob('*'))[10]
print(random_nii_file)

def open_nii_file(path):
    f = nib.load(path)
    segmentations = f.get_fdata()[:, ::-1, ::-1].transpose(2, 1, 0)
    return segmentations

nii_segments = open_nii_file(random_nii_file)
print(nii_segments.shape, "=> (num_slices, height, width)")
plt.imshow(nii_segments[123,:,:],cmap='bone')

In [ ]:
# np.unique(nii_segments[slice_number])
print("[background, *, vertebra]")
print("here 4. denotes that the slice is a part of C4 vertebra")
np.unique(nii_segments[100])

In [ ]:
f"We only have {len(list(paths['train_nifti_segments'].glob('*')))} nii files / {len(train_df)} studies"

In [ ]:
def add_nii_segment_path(uid):
    base_path = paths['train_nifti_segments']
    # path if exists else None
    path = base_path/(uid+'.nii')
    if path.exists():
        return path
    return None

train_df['nii_segments_path'] = train_df['StudyInstanceUID'].map(add_nii_segment_path)

In [ ]:
def get_dcm_images(path):
    paths = list(path.glob('*'))
    paths.sort(key=lambda x:int(x.stem)) # sort based on slice index which is the filename: index.dcm
    data = [pydicom.dcmread(f) for f in paths]
    images = [apply_voi_lut(dcm.pixel_array, dcm) for dcm in data]
    return images

In [ ]:
def get_nii_segments(path):
    f = nib.load(path)
    segmentations = f.get_fdata()[:, ::-1, ::-1].transpose(2, 1, 0)
    return segmentations

In [ ]:
sample_idx = 99
dcm_images = get_dcm_images(train_df['segment_path'][sample_idx])
nii_segments = get_nii_segments(train_df['nii_segments_path'][sample_idx])
print((len(dcm_images), *dcm_images[0].shape), nii_segments.shape)

In [ ]:
def plot_slice(idx, dcm_images=dcm_images, nii_segments=nii_segments):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.axis('off'); ax2.axis('off')
    fig.suptitle(f'Slice {idx}')
    ax1.imshow(dcm_images[idx], cmap='bone')
    ax2.imshow(nii_segments[idx,:,:], cmap='bone')

for i in range(123,128):
    plot_slice(i)

In [ ]:
uids_with_segments = train_df[train_df['nii_segments_path'].notnull()]
uids_with_segments.head()

In [ ]:
def check_reverse_required(path):
    paths = list(path.glob('*'))
    paths.sort(key=lambda x:int(x.stem))
    z_first = pydicom.dcmread(paths[0]).get("ImagePositionPatient")[-1]
    z_last = pydicom.dcmread(paths[-1]).get("ImagePositionPatient")[-1]
    if z_last < z_first:
        return False
    return True

In [ ]:
checks = train_df['segment_path'].progress_map(check_reverse_required)

In [ ]:
train_df['reverse_required'] = checks

In [ ]:
Counter(checks)

In [ ]:
indices_where_reverse_required = [i for i,req in (checks.reset_index()).values if req is True]
print(indices_where_reverse_required)

In [ ]:
train_bbox = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv")

In [ ]:
train_bbox.head()

In [ ]:
# Visualize image with bounding box

# for uid, path1, rev in tqdm(train_df.loc[101:110,['StudyInstanceUID','segment_path','reverse_required']].values):
# path1= f'../input/rsna-2022-cervical-spine-fracture-detection/train_images/{instance_uid}/.dcm'
dcm_images = get_dcm_images(path)
dcm_np = np.array(dcm_images)
def visualize_bbox_image(instance_uid,slice_num):
    path1= f'../input/rsna-2022-cervical-spine-fracture-detection/train_images/{instance_uid}/{slice_num}.dcm'
    path =path1
    dcm = pydicom.dcmread(path1)
    image = apply_voi_lut(dcm.pixel_array, dcm)
    info = train_bbox[(train_bbox.StudyInstanceUID==instance_uid) & (train_bbox.slice_number==slice_num)]
                      
    bbox = patches.Rectangle((float(info.x), float(info.y)), float(info.width),
                             float(info.height), linewidth=3, edgecolor='r', facecolor='none')
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(1, 1, 1)
    plt.imshow(image, cmap='bone')
    ax.add_patch(bbox)
    plt.title(f'ID: {instance_uid}, Slice: {slice_num}',weight='bold', size=12, y=1.04)
    ax.axis('off')
    plt.show()

In [ ]:
for uid, slice_number in tqdm(train_bbox.loc[101:110,['StudyInstanceUID','slice_number']].values):
    max_slice=0
    if slice_number>max_slice:
        max_slice=slice_number
        visualize_bbox_image(uid,slice_number)

In [ ]:
base_path = "../input/rsna-2022-cervical-spine-fracture-detection"

def plot_fracture(slice_num,bbox_id,ax_id1,ax_id2):
    file = pydicom.dcmread(f"{base_path}/train_images/{bbox_id}/{slice_num}.dcm")
    img = apply_voi_lut(file.pixel_array, file)
    info = train_bbox[(train_bbox['StudyInstanceUID']==bbox_id)&(train_bbox['slice_number']==slice_num)]
    rect = patches.Rectangle((float(info.x), float(info.y)), float(info.width), float(info.height), linewidth=3, edgecolor='r', facecolor='none')

    axes[ax_id1,ax_id2].imshow(img, cmap="bone")
    axes[ax_id1,ax_id2].add_patch(rect)
    axes[ax_id1,ax_id2].set_title(f"ID:{bbox_id}, Slice: {slice_num}", fontsize=20, weight='bold',y=1.02)
    axes[ax_id1,ax_id2].axis('off')

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18,18))
plot_fracture(146,'1.2.826.0.1.3680043.10051',0,0)
plot_fracture(151,'1.2.826.0.1.3680043.10678',0,1)
plot_fracture(174,'1.2.826.0.1.3680043.10678',1,0)
plot_fracture(90,'1.2.826.0.1.3680043.10732',1,1)